# $M/HYPO_r/1$ queue
Arrival rate $\lambda$

perpareTransfer and sendTrytes service rates are $\mu_1$ and $\mu_2$, respectively.

$Q = \begin{pmatrix}
B_{00} &B_{01}  & 0 & 0 & 0 & \ldots\\ 
B_{10} & A_{1} & A_{2} & 0 & 0 &\dots \\ 
 0& A_{0} &  A_{1} & A_{2}  & 0 &\ldots \\ 
 0&  0& A_{0} &  A_{1}  & A_{2} & \\ 
 0&  0&  0& A_{0} &  A_{1}  & \ldots\\ 
 \vdots& \vdots & \vdots & \vdots & \vdots & \ddots
\end{pmatrix}$

- $A_0$: Transitions from state (k, r) to state (k − 1, 1), the transit rate of the last service phase.
- $A_1$: The superdiagonal elements of the matrices $A_1$ represent phase completion to state (k + 1, i ). It follows that the only nonzero elements in this matrix are the diagonal elements The diagonal elements are set equal to the negated sum of the off-diagonal elements of Q. All other at rate rμ in service phase i < r at level k > 0, i.e., transitions from state (k, i ) to state (k, i + 1).
- $A_2$: The matrices A2 represent arrivals at rate λ which can occur during service at any phase i at any level k > 0, i.e., transitions from state (k, i ) which are all equal to λ.
- $B_{01}$: A 1 × r row matrix all of whose elements are zero except for the first which is equal to $\lambda$. This is the rate of transition from state (0, 0) to state (1, 1) and corresponds to an arrival to an empty system.
- $B_{10}$: A r × 1 column matrix all of whose elements are zero except for the last which is equal to $\mu_r$. This is the rate of transition from state (1, r) to state (0, 0) and corresponds to the complete service termination of the only customer in the system.
- $B_{00}$: A 1×1 matrix whose nonzero element is $-\lambda$ and reflects the fact that the sum across the first row must be zero.

$A_0=\begin{pmatrix}
0 & 0  & 0 & 0 & \ldots & 0\\ 
0 & 0 & 0 & \ldots & \ldots &0 \\ 
 0& 0 &  0 & 0  & \ldots &0 \\ 
 0&  0& 0 &  0  & \ldots & 0\\ 
 \vdots& \vdots & \vdots & \vdots &\ddots & \vdots\\ 
 \mu_r & 0 & 0 & 0 & \ldots & 0
\end{pmatrix}$,$A_2=\lambda I_r$, $A_1 = \begin{pmatrix}
-\lambda-\mu_1 & \mu_1  & 0 & 0 & \ldots & 0\\ 
0 & -\lambda-\mu_2 & \mu_2 & 0 & \ldots &0 \\ 
0 & 0 &  -\lambda-\mu_3 & \mu_3  & \ldots &0 \\ 
\vdots& \vdots & \vdots & \ddots &\ddots & \vdots\\ 
0&  0& 0 &  0  & \vdots & \mu_{r-1}\\
 0 & 0 & 0 & 0 & \ldots & -\lambda-\mu_r
\end{pmatrix}$

$B_{00}=-\lambda$,$B_{01}=\begin{pmatrix}
\lambda & 0 & 0 & \ldots & 0
\end{pmatrix}$,$B_{10}=\begin{pmatrix}
0\\ 
0\\  
\vdots\\ 
0 \\
\mu_r
\end{pmatrix}$


In [41]:
import numpy as np
from numpy import kron,eye,multiply

In [69]:
## H_2 Arrival Process:
alpha1 = 0.4
alpha2 = 0.6
lambda1 = 1
lambda2 = 2
T = [[-lambda1, 0], 
     [0, -lambda2]]
T0 = [[lambda1],[lambda2]]
xi = [alpha1, alpha2]

In [70]:
## E_3 Service Process:
mu1=4
mu2=8
mu3=8
S = [[-mu1, mu1, 0],
     [0, -mu2, mu2],
     [0,0, -mu3]]
S0 = [[0],[0],[mu3]]
sigma = [1,0,0]

In [71]:
## Block Submatrices for all types of queues:
ra = len(T[0])
rs = len(S[0])
A0 = kron(eye(ra), multiply(S0,sigma))
A1 = kron(T, eye(rs)) + kron(eye(ra), S)
A2 = kron(multiply(T0,xi), eye(rs))
B00 = T
B01 = kron(multiply(T0,xi),sigma)
B10 = kron(eye(ra),S0)
l = len(B00[0])
r = len(A0[0])

# Form Neuts’ R matrix  
V = multiply(A2, np.linalg.inv(A1))
W = multiply(A0, np.linalg.inv(A1))
R = np.zeros((r,r)) 
Rbis = -V - multiply(multiply(R,R), W)
n = 1
err = 1.0e-10
while (np.linalg.norm(R-Rbis,1) > err):
    R = Rbis
    Rbis = -V - multiply(multiply(R,R), W)
    n = n+1
R = Rbis

0.19999999999999998


In [61]:
# M/HYPO2/1

## M Arrival Process:
lambda1 = 2
T = [-lambda1]
T0 = [lambda1]
xi = [1]

## E_2 Service Process:
mu1=8
mu2=4
S = [[-mu1, mu1],
     [0, -mu2]]
S0 = [[0],[mu2]]
sigma = [1,0]

In [62]:
## Block Submatrices for all types of queues:
ra = len(T)
rs = len(S[0])
A0 = kron(eye(ra), multiply(S0,sigma))
A1 = kron(T, eye(rs)) + kron(eye(ra), S)
A2 = kron(multiply(T0,xi), eye(rs))
B00 = T
B01 = kron(multiply(T0,xi),sigma)
B10 = kron(eye(ra),S0)
l = len(B00)
r = len(A0[0])

In [63]:
print("A0:",A0,'\n',"A1:",A1,'\n',"A2:",A2,'\n',"B00:",B00,'\n',"B01:",B01,'\n',"B10:",B10)

A0: [[0. 0.]
 [4. 0.]] 
 A1: [[-10.   8.]
 [  0.  -6.]] 
 A2: [[2. 0.]
 [0. 2.]] 
 B00: [-2] 
 B01: [2 0] 
 B10: [[0.]
 [4.]]


In [64]:
# Form Neuts’ R matrix  
V = multiply(A2, np.linalg.inv(A1))
W = multiply(A0, np.linalg.inv(A1))
R = np.zeros((r,r)) 
Rbis = -V - multiply(multiply(R,R), W)
n = 1
err = 1.0e-10
while (np.linalg.norm(R-Rbis,1) > err):
    print(np.linalg.norm(R-Rbis,1))
    R = Rbis
    Rbis = -V - multiply(multiply(R,R), W)
    n = n+1
R = Rbis

0.3333333333333333


In [68]:
R

array([[0.2       , 0.        ],
       [0.        , 0.33333333]])

In [66]:
V

array([[-0.2       , -0.        ],
       [-0.        , -0.33333333]])

In [67]:
W

array([[-0., -0.],
       [-0., -0.]])

In [74]:
## M Arrival Process:
lambda1 = 1
T = [-lambda1]
T0 = [lambda1]
xi = [1]

## E_2 Service Process:
r = 3
mu = 1.5
mu1,mu2,mu3 = r*mu, r*mu, r*mu

S = [[-mu1, mu1,0],
     [0, -mu2,mu2],
    [0,0,-mu3]]
S0 = [[0],[0],[mu3]]
sigma = [1,0,0]

## Block Submatrices for all types of queues:
ra = len(T)
rs = len(S[0])
A0 = kron(eye(ra), multiply(S0,sigma))
A1 = kron(T, eye(rs)) + kron(eye(ra), S)
A2 = kron(multiply(T0,xi), eye(rs))
B00 = T
B01 = kron(multiply(T0,xi),sigma)
B10 = kron(eye(ra),S0)
l = len(B00)
r = len(A0[0])

In [87]:
A2

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [81]:
# Form Neuts’ R matrix  
V = multiply(A2, np.linalg.inv(A1))
W = multiply(A0, np.linalg.inv(A1))
R = np.zeros((r,r)) 
Rbis = -V - multiply(multiply(R,R), W)
n = 1
err = 1.0e-10
while (np.linalg.norm(R-Rbis,1) > err):
    print(np.linalg.norm(R-Rbis,1))
    R = Rbis
    Rbis = -V - multiply(multiply(R,R), W)
    n = n+1
R = Rbis

0.18181818181818182


In [84]:
np.linalg.inv(A1)

array([[-0.18181818, -0.14876033, -0.121713  ],
       [-0.        , -0.18181818, -0.14876033],
       [-0.        , -0.        , -0.18181818]])

In [85]:
multiply(A2, np.linalg.inv(A1))

array([[-0.18181818, -0.        , -0.        ],
       [-0.        , -0.18181818, -0.        ],
       [-0.        , -0.        , -0.18181818]])

In [86]:
A2

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])